In [3]:
import torch
from transformers import AutoTokenizer
import json
import requests
from transformers import LayoutLMModel
from pytorch_lightning import LightningModule
import torch.nn as nn
from infer.model.loss import BboxLoss
from infer.model.spade_model import RelationTagger
import json

# jsonfile =  json.load(open('./notebook/json_file/cccd_test.jsonl')) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
def get_bbox(jsonl_file):
    w = jsonl_file['img_sz']['width']
    h = jsonl_file['img_sz']['height']
    bboxes = [[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in jsonl_file['coord']]
    # return  torch.tensor(bboxes)
    return bboxes

In [7]:
from dataModel.datamd_ import DpDataSet
import numpy as np
data = DpDataSet('./notebook/json_file/test_shuffe.jsonl')

data[0]['maps_tensor'].shape, 

i = 0
for item in data[0]['maps']:
    for idx in item: 
        i = i + 1
i, data[0]['maps_tensor'].shape

(127, torch.Size([1, 127]))

In [8]:
# m = np.array(maps.cpu())
# m = m.tolist()
# box_index =  np.unique(m).tolist()
# [[item]*m.count(item) for item in box_index]
# input_ids.shape
from functools import lru_cache
@lru_cache
def load_model():

    return LayoutLMModel.from_pretrained(
                "microsoft/layoutlm-base-uncased", local_files_only=True)


In [9]:
from torch import Tensor
model =  load_model()
# maps
input_ids = data[0]['input_ids']
bbox = data[0]['bbox']
attention_mask = data[0]['attention_mask']
token_type_ids = data[0]['token_type_ids']
maps = data[0]['maps']
maps_tensor = data[0]['maps_tensor']

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing LayoutLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
outputs = model(input_ids=input_ids,bbox=bbox,attention_mask=attention_mask,token_type_ids=token_type_ids)
outputs.last_hidden_state.shape

reduce_size = 256

ln = nn.Linear(model.config.hidden_size, reduce_size)

last_hidden_state = ln(outputs.last_hidden_state)
last_hidden_state.shape

zeros_temp = torch.zeros(reduce_size)
# m = nn.BatchNorm1d(last_hidden_state.shape[1] , affine=False)
# last_hidden_state, m (last_hidden_state).shape

In [11]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action = 'ignore')
r = ['aăâbcdđeêghiklmnoôơpqrstuưvxy']
# model = Word2Vec(sentences=data[0]['text'], vector_size=100, window=5, min_count=1, workers=4,  sg= 1)


srt = [['hooman', 'interface', 'computer'],
  ['survey', 'user', 'computer', 'system', 'response', 'time'],
  ['eps', 'user', 'interface', 'system'],
  ['system', 'human', 'system', 'eps'],
  ['user', 'response', 'time'],
  ['trees'],
  ['graph', 'trees'],
  ['graph', 'minors', 'trees'],
  ['graph', 'minors', 'survey']]

model = Word2Vec(sentences=[[item] for item in data[0]['text']] , vector_size=100, window=5, min_count=1, workers=4,  sg= 1)

# common_texts

len(model.wv) #, common_texts
# model.wv['hooman']

[[item] for item in data[0]['text']] , data[0]['text'],len(data[0]['text']) 


def wordemd_loss (text):
  model = Word2Vec(sentences=[[item] for item in text] , vector_size=100, window=5, min_count=1, workers=4,  sg= 1)
  return model.wv

In [41]:
from numpy import linalg as LA

b = 0 + wordemd_loss(data[0]['text'])[0] +  wordemd_loss(data[0]['text'])[1]

LA.norm(b)

0.08055088

In [ ]:

# def tranfer_maps(maps: Tensor):
#     m = []
#     maps = torch.squeeze(maps, 0)
#     uniq, count= maps.unique(return_counts=True)
#     for i in range(len(count)):
#         m.append(torch.stack([uniq[i]]*count[i]))
#     return m


def reduce_shape_old(last_hidden_state, maps):
        device = last_hidden_state.device
        reduce = []
        for g_token in maps:
            ten = zeros_temp.to(device)
            for ele in g_token:
                # print(g_token[0],'-----')
                ten += last_hidden_state[0][g_token[0]]
                # print(ele,'----')
            ten = ten/len(g_token)
            # print(len(g_token))
            reduce.append(ten)
        reduce = torch.stack(reduce).to(device)
        return reduce



reduce_old = reduce_shape_old(last_hidden_state, maps)




rel_s = RelationTagger(
            hidden_size=reduce_size,
            n_fields=3,
        )

S = rel_s(reduce_old.unsqueeze(0))
# tranfer_maps(maps)
# maps = 
# reduce_shape(last_hidden_state, tranfer_maps(maps) ) 
reduce_old.shape, np.shape(maps), S.shape


In [ ]:

def reduce_shape(last_hidden_state, maps: Tensor):
    maps = torch.squeeze(maps, 0)
    uniq, count= maps.unique(return_counts=True)
    device = last_hidden_state.device
    reduce = []
    for idx,  g_token in enumerate (uniq):
        ten = zeros_temp.to(device)
        for j in range (count[idx]):
            # print(g_token.numpy(),'-----')
            ten += last_hidden_state[0][g_token.numpy()]
        # print(g_token, idx)
        ten = ten/count.detach().numpy()[idx]
        # print(count.detach().numpy()[idx])
        reduce.append(ten)
    reduce = torch.stack(reduce).to(device)
    return reduce
reduce = reduce_shape(last_hidden_state, maps_tensor)
# # maps

# with torch.no_grad():
#   b =  torch.mean(reduce -reduce_old)
#   print(b)

reduce

In [ ]:
def tranfer_maps(maps: Tensor):
    m = []
    maps = torch.squeeze(maps, 0)
    uniq, count= maps.unique(return_counts=True)
    for i in range(len(count)):
        m.append(torch.stack([uniq[i]]*count[i]).tolist())
    return m



a = reduce_shape_old(last_hidden_state, maps) - reduce_shape_old(last_hidden_state,maps )
with torch.no_grad():
    
    print(torch.mean(a))

# tranfer_maps(maps_tensor), maps

In [ ]:
# With Learnable Parameters
# m = nn.BatchNorm2d(100)
# Without Learnable Parameters
m = nn.BatchNorm2d(100, affine=False)
input = torch.randn(20, 100, 35, 45)
output = m(input)

output.shape

In [ ]:
input.shape

In [ ]:

# %reload_ext autoreload
from models.layout_parsing import LitBaseParsing
%load_ext autoreload
%autoreload 2
from models.base import  LitLayoutParsing
model = LitBaseParsing()

In [ ]:
# totalGraph =  model(input_ids, attention_mask, token_type_ids, bbox,maps)

# totalGraph.shape
dummy_input = torch.zeros(3,126)
dummy_input[1]

In [ ]:
# dummy_input = torch.zeros_like(input_ids).cuda()
# dummy_boxes = torch.zeros_like(bbox).cuda()
# dummy_maps = torch.zeros_like(maps).cuda()
import torch
dummy_input = torch.zeros(1,196).cuda()
dummy_boxes = torch.zeros(1,196,4).cuda()
dummy_maps = torch.zeros(1,196).cuda()
# totalGraph[0].shape
# S = totalGraph[0].unsqueeze(0)
# G = totalGraph[1].unsqueeze(0)
# dummy_maps.shape
# dummy_maps.unsqueeze(0).shape

In [ ]:
import onnxruntime as ort
import numpy as np


providers = [
            # 'TensorrtExecutionProvider',
                     'CUDAExecutionProvider',
                     'CPUExecutionProvider']

ort_sess = ort.InferenceSession(
                                path_or_bytes = 'resources/onnx/LitLP_06-03-2023_18-46-25_fp32.onnx'
                                , providers= providers)

In [ ]:
input_name = ort_sess.get_inputs()
# output_name = ort_sess.get_outputs()[0].name
input_name[3].name


# outputs = ort_sess.run(None, 
#                         {
#                             'input_1': input_ids.numpy()
#                             'input_2': input_ids.numpy()
#                             'input_3': input_ids.numpy()
#                             'input_4': input_ids.numpy()
#                             'input_5': input_ids.numpy()
#                         }
#                         )
